Primero importemos las librerías que usaremos a lo largo del experimento, además de marcar que el solver no nos muestre el progreso, definir un tiempo límite para la ejecución de las funciones y una semilla para que los experimentos sean reproducibles

In [345]:
import cvxopt
import time
import numpy as np
from sklearn.model_selection import train_test_split
from math import log, floor, sqrt
from func_timeout import func_set_timeout, FunctionTimedOut

cvxopt.solvers.options['show_progress'] = False
timelimit = 100
semilla = 33

Ahora consideremos una función que resuelve un problema cuadrático. Esta función fue obtenida del recurso entregado en el enunciado de la tarea. Para los problemas que estamos considerando esta función tiene errores numéricos cuando la solución no existe puesto que se empieza a ir a infinito antes de que la matriz KKT sea singular, por lo que en caso de fallar devolveremos que la solución no existe (lo implementaremos con una excepción)

In [346]:
def solver(P, q, G, h):
    args = [cvxopt.matrix(P), cvxopt.matrix(q), cvxopt.matrix(G), cvxopt.matrix(h)]
    try:
        sol = cvxopt.solvers.qp(*args)
        if 'optimal' not in sol['status']:
            return None
        return np.array(sol['x']).reshape((P.shape[1],))
    except ValueError as e:
        if "domain error" in str(e):
            return None
        else:
            raise

Ahora consideremos crear una función que me divida los datos en datos de entrenamiento, de testeo y de validación según una proporción dada

In [347]:
def splitter(X, Y, train = 0.8, validate = 0.1, test = 0.1):
    X_train, X_aux, Y_train, Y_aux = train_test_split(X, Y, train_size = train, random_state = semilla)
    X_val, X_test, Y_val, Y_test = train_test_split(X_aux, Y_aux, train_size = validate/(validate + test), random_state = semilla)
    return X_train, X_val, X_test, Y_train, Y_val, Y_test

Consideremos una función que lea los datos en función del test a considerar y que además los preprocese agregando la columna adicional con unos para que el separador sea afín.

In [348]:
def leer(test):
    X = np.load('datos_T1/data/test_' + str(test) + '/X.npy')
    X = np.append(X, np.ones((X.shape[0], 1)), axis=1)
    Y = np.load('datos_T1/data/test_' + str(test) + '/Y.npy')
    return X, Y

Ahora procedamos a resolver el problema

Para el problema (hard-SVM) notemos que $$\lVert w \rVert_2^2 = w^TIw$$ donde $I$ es la matriz identidad, por lo que el problema (hard-SVM) se puede escribir en forma cuadrática como 


\begin{aligned}
&\min_{w \in \mathbb{R}^d} \frac{1}{2} w^TIw \\
&\text{s.a. } -Y_i\sum_{j=1}^d X_{ij}\cdot w_j \leq -1 \quad \forall i \in [n]
\end{aligned}

Y para el formato descrito por el solver tenemos que $P = I$, $q = 0$, $G = XI(-Y_i)$, $h = -\vec{1}$


In [349]:
@func_set_timeout(timelimit)
def hardSVMsolver(X, Y):
    n, d = X.shape
    P = np.eye(d, dtype = float)
    q = np.zeros(d, dtype = float)
    G = (X * (-Y[:, np.newaxis]))
    h = np.full(n, -1, dtype = float)
    w = solver(P, q, G, h)
    return w

Para el problema (soft-SVM) consideremos las primeras $d$ entradas del vector variable como $w$ y las siguientes $n$ entradas como $\xi$. El problema (soft-SVM) se puede escribir en forma cuadrática como 


\begin{aligned}
&\min_{w \in \mathbb{R}^d} &\frac{\lambda}{2} w^TIw + \sum_{i=1}^n \xi_i &\\
&\text{s.a. } &-Y_i\sum_{j=1}^d X_{ij}\cdot w_j - \xi_i \leq -1 & \ \ \forall i \in [n] \\
& & -\xi_i \leq 0 & \ \ \forall i \in [n]
\end{aligned}

Y para el formato descrito por el solver tenemos que $P = \binom{\lambda \cdot I_{d,d} \ 0_{d,n}}{0_{n,d} \ \ \ \ \ 0_{n,n}}$, $q = \binom{0}{\vec{1}_n}$, $G = \binom{XI(-Y_i) \ -I_n}{0_{n, d} \ \ \ \ \ \ \ \ \ \ \ \ -I_n}$, $h = \binom{-\vec{1}_n}{\vec{0}_n}$.

Además debemos separar los datos en datos de entrenamiento, datos de validación y datos de prueba, de tal forma de escoger el $\lambda$ adecuado de forma algorítmica, para esto vamos a diseñar una rutina que separe los datos según una cierta proporción.

Y lo implementamos en conjunto con la búsqueda automática del $\lambda$ óptimo

In [350]:
def softSVM(X, Y, Lambda):
    n, d = X.shape
    P = np.zeros((d+n, d+n), dtype = float)
    P[:d, :d] = Lambda*np.eye(d, dtype = float)
    
    q = np.zeros(d+n, dtype = float)
    q[d:] = 1.0
    
    G = np.zeros((2*n, d+n), dtype = float)
    G[:n, :d] = -Y[:, np.newaxis] * X
    G[:n, d:] = -np.eye(n)
    G[n:, d:] = -np.eye(n)
    
    h = np.zeros(2*n, dtype = float)
    h[:n] = -1.0
    
    return solver(P, q, G, h)[:d]

@func_set_timeout(timelimit)
def softSVMsolver(X, Y):
    n, d = X.shape
    X1, X2, X3, Y1, Y2, Y3 = splitter(X, Y)
    best = n+1
    wbest = np.zeros(d)
    for exp in range(0, 11):
        Lambda = 2**(-exp)
        w = softSVM(X1, Y1, Lambda)
        v = (Lambda/2.0)*np.dot(w, w) + sum(max(0.0, 1.0 - np.dot(w, X2[i])*Y2[i]) for i in range(len(X2)))
        if v < best:
            best = v
            wbest = w
    risk = 100*(sum(1 for i in range(len(X3)) if Y3[i] *np.dot(wbest, X3[i]) < 1))/len(X3)
    return wbest, risk
        

Primero veamos para cada caso de prueba si son linealmente separables.

In [351]:
for i in range(1, 6):
    X, Y = leer(i)
    print(f"Estamos resolviendo el caso {i} que tiene {X.shape[0]} datos y {X.shape[1] - 1} características")
    tstart = time.time()
    try:
        w = hardSVMsolver(X, Y)
        tfinish = time.time()
        print(f"Se demoró {(tfinish - tstart):.2f} segundos y ", end = '')
        if w is None:
            print("no existe un separador afín")
        else:
            print(f"encontró un separador de margen {(1/np.dot(w, w)):.2f}")
    except FunctionTimedOut:
        print(f"Se está demorando más de {timelimit} segundos así que abortaremos el cálculo")
    print()

Estamos resolviendo el caso 1 que tiene 100 datos y 20 características
Se demoró 0.05 segundos y encontró un separador de margen 0.83

Estamos resolviendo el caso 2 que tiene 800 datos y 100 características
Se demoró 0.97 segundos y no existe un separador afín

Estamos resolviendo el caso 3 que tiene 2000 datos y 300 características
Se demoró 0.40 segundos y encontró un separador de margen 0.74

Estamos resolviendo el caso 4 que tiene 3000 datos y 500 características
Se demoró 3.97 segundos y no existe un separador afín

Estamos resolviendo el caso 5 que tiene 5000 datos y 1000 características
Se demoró 5.42 segundos y encontró un separador de margen 1.27



Notemos que el algoritmo fue capaz de encontrar el óptimo muy rápido (menos de 6 segundos) y que los casos separables son el 1, 3 y 5. Este algoritmo fue capaz de resolver el caso con 5000 datos y 1000 características en unos 5 segundos, por lo que una extrapolación del tiempo usado (al ojo) es que sería capaz de resolver algo con 50000 datos y 3000 características en un tiempo razonable

Y ahora resolvamos los casos de prueba con (Soft-SVM)

In [352]:
for i in range(1, 6):
    X, Y = leer(i)
    print(f"Estamos resolviendo el caso {i} que tiene {X.shape[0]} datos y {X.shape[1] - 1} características")
    tstart = time.time()
    try:
        w, risk = softSVMsolver(X, Y)
        tfinish = time.time()
        print(f"Se demoró {(tfinish - tstart):.2f} segundos y ", end = '')
        print(f"encontró un separador de margen {(1/np.dot(w, w)):.2f} y en los datos de testing acertó un {risk}%")
    except FunctionTimedOut:
        print(f"Se está demorando más de {timelimit} segundos así que abortaremos el cálculo")
    print()

Estamos resolviendo el caso 1 que tiene 100 datos y 20 características
Se demoró 1.11 segundos y encontró un separador de margen 1.75 y en los datos de testing acertó un 50.0%

Estamos resolviendo el caso 2 que tiene 800 datos y 100 características
Se demoró 12.50 segundos y encontró un separador de margen 4.54 y en los datos de testing acertó un 41.25%

Estamos resolviendo el caso 3 que tiene 2000 datos y 300 características
Se está demorando más de 100 segundos así que abortaremos el cálculo

Estamos resolviendo el caso 4 que tiene 3000 datos y 500 características
Se está demorando más de 100 segundos así que abortaremos el cálculo

Estamos resolviendo el caso 5 que tiene 5000 datos y 1000 características
Se está demorando más de 100 segundos así que abortaremos el cálculo



Ahora obviamente siempre va a encontrar un separador (aunque sea uno muy malo). Notemos que acierta bastante poco en los casos que no ha visto, pero se debe en una parte a la naturaleza de los datos y en otra muy grande a la forma en que se escoje el $\lambda$ óptimo (de la cual no estoy muy de acuerdo pero prefiero seguir el enunciado). Notemos que los 2 primeros casos anduvieron bien en tiempo, pero el tercero se demoró más de 100 segundos, por lo que se espera que este algoritmo sea capaz de encontrar el óptimo para un conjunto de unos 1000 datos y 150 características.

Ahora preprocesemos los datos con una matriz aleatoria tal como el algoritmo que diseñamos en la pregunta 4 y veamos la capacidad de dicho algoritmo de encontrar un separador

In [353]:
def WinRateLowDim(X, Y, k, tries):
    success = 0
    for _ in range(tries):
        A = np.random.randn(k, X.shape[1])
        newX = (1.0/sqrt(k))*np.dot(X, A.T)
        w = hardSVMsolver(newX, Y)
        if w is not None:
            success += 1
    return 100*success/tries

In [354]:
for i in range(1, 6):
    X, Y = leer(i)
    k = floor(81*log(16*X.shape[0]**2)/4)
    print(f"En el caso {i} hay {X.shape[0]} datos y reduciremos la dimensión de los datos de {X.shape[1]} a {k}")
    try:
        print(f"Pudimos separar para el {WinRateLowDim(X, Y, k, 10)}% de las matrices")
    except FunctionTimedOut:
        print("Las iteraciones se están demorando demasiado, así que abortamos el cálculo")
    print()

En el caso 1 hay 100 datos y reduciremos la dimensión de los datos de 21 a 242
Pudimos separar para el 100.0% de las matrices

En el caso 2 hay 800 datos y reduciremos la dimensión de los datos de 101 a 326
Pudimos separar para el 0.0% de las matrices

En el caso 3 hay 2000 datos y reduciremos la dimensión de los datos de 301 a 363
Pudimos separar para el 100.0% de las matrices

En el caso 4 hay 3000 datos y reduciremos la dimensión de los datos de 501 a 380
Pudimos separar para el 0.0% de las matrices

En el caso 5 hay 5000 datos y reduciremos la dimensión de los datos de 1001 a 401
Pudimos separar para el 0.0% de las matrices



Para el caso 1 y 3 las matrices eran separables y el 100% encontró separadores, sin embargo el caso 5 también era separable y no logró encontrar un separador nunca. En los casos no separables siguió sin poder encontrar un separador. Sin embargo la cota obtenida en la pregunta 4 es asintóticamente logarítmica pero tiene unas constantes muy grandes y además depende de $n$ que es el número de datos y no del $k$ que es la dimensión de los datos, por lo que si observamos bien hay casos en los que la "reducción" de dimensionalidad hace la dimensionalidad más grande. Veamos que ocurre si definimos una reducción proporcional a $k$ de la dimensionalidad, probemos con $0.8$ de la dimensión 

In [355]:
for i in range(1, 6):
    X, Y = leer(i)
    k = floor(0.8*X.shape[1])
    print(f"En el caso {i} hay {X.shape[0]} datos y reduciremos la dimensión de los datos de {X.shape[1]} a {k}")
    try:
        print(f"Pudimos separar para el {WinRateLowDim(X, Y, k, 10)}% de las matrices")
    except FunctionTimedOut:
        print("Las iteraciones se están demorando demasiado, así que abortamos el cálculo")
    print()

En el caso 1 hay 100 datos y reduciremos la dimensión de los datos de 21 a 16
Pudimos separar para el 10.0% de las matrices

En el caso 2 hay 800 datos y reduciremos la dimensión de los datos de 101 a 80
Pudimos separar para el 0.0% de las matrices

En el caso 3 hay 2000 datos y reduciremos la dimensión de los datos de 301 a 240
Pudimos separar para el 0.0% de las matrices

En el caso 4 hay 3000 datos y reduciremos la dimensión de los datos de 501 a 400
Pudimos separar para el 0.0% de las matrices

En el caso 5 hay 5000 datos y reduciremos la dimensión de los datos de 1001 a 800
Pudimos separar para el 0.0% de las matrices



En este caso se puede ver que pese a que la reducción de la dimensión no fue mucha, casi nunca era capaz de encontrar un separador. Quizás si el tiempo se reduce en lo suficiente como para probar demasiadas matrices distintas podría llegar a ser conveniente, ¡veámoslo!

Estudiaremos los tiempos de ejecución tanto para (Hard-SVM) como para (Soft-SVM), y para el $k$ obtenido en la pregunta 4 y para el $k$ proporcional

In [356]:
for i in range(1, 6):
    X, Y = leer(i)
    k = floor(81*log(16*X.shape[0]**2)/4)
    A = np.random.randn(k, X.shape[1])
    newX = (1.0/sqrt(k))*np.dot(X, A.T)
    print(f"En el caso {i} hay {X.shape[0]} datos y reduciremos la dimensión de los datos de {X.shape[1]} a {k}")
    try:
        tstart = time.time()
        w = hardSVMsolver(newX, Y)
        tfinish = time.time()
        print(f"Se demoró {(tfinish - tstart):.2f} segundos")
    except FunctionTimedOut:
        print(f"Se demoró más de {timelimit} segundos")
    print()

En el caso 1 hay 100 datos y reduciremos la dimensión de los datos de 21 a 242
Se demoró 0.07 segundos

En el caso 2 hay 800 datos y reduciremos la dimensión de los datos de 101 a 326
Se demoró 0.09 segundos

En el caso 3 hay 2000 datos y reduciremos la dimensión de los datos de 301 a 363
Se demoró 0.23 segundos

En el caso 4 hay 3000 datos y reduciremos la dimensión de los datos de 501 a 380
Se demoró 1.71 segundos

En el caso 5 hay 5000 datos y reduciremos la dimensión de los datos de 1001 a 401
Se demoró 1.19 segundos



In [357]:
for i in range(1, 6):
    X, Y = leer(i)
    k = floor(0.8*X.shape[1])
    A = np.random.randn(k, X.shape[1])
    newX = (1.0/sqrt(k))*np.dot(X, A.T)
    print(f"En el caso {i} hay {X.shape[0]} datos y reduciremos la dimensión de los datos de {X.shape[1]} a {k}")
    try:
        tstart = time.time()
        w = hardSVMsolver(newX, Y)
        tfinish = time.time()
        print(f"Se demoró {(tfinish - tstart):.2f} segundos")
    except FunctionTimedOut:
        print(f"Se demoró más de {timelimit} segundos")
    print()

En el caso 1 hay 100 datos y reduciremos la dimensión de los datos de 21 a 16
Se demoró 0.09 segundos

En el caso 2 hay 800 datos y reduciremos la dimensión de los datos de 101 a 80
Se demoró 0.19 segundos

En el caso 3 hay 2000 datos y reduciremos la dimensión de los datos de 301 a 240
Se demoró 0.91 segundos

En el caso 4 hay 3000 datos y reduciremos la dimensión de los datos de 501 a 400
Se demoró 0.92 segundos

En el caso 5 hay 5000 datos y reduciremos la dimensión de los datos de 1001 a 800
Se demoró 5.57 segundos



In [358]:
for i in range(1, 6):
    X, Y = leer(i)
    k = floor(81*log(16*X.shape[0]**2)/4)
    A = np.random.randn(k, X.shape[1])
    newX = (1.0/sqrt(k))*np.dot(X, A.T)
    print(f"En el caso {i} hay {X.shape[0]} datos y reduciremos la dimensión de los datos de {X.shape[1]} a {k}")
    try:
        tstart = time.time()
        w = softSVMsolver(newX, Y)
        tfinish = time.time()
        print(f"Se demoró {(tfinish - tstart):.2f} segundos")
    except FunctionTimedOut:
        print(f"Se demoró más de {timelimit} segundos")
    print()

En el caso 1 hay 100 datos y reduciremos la dimensión de los datos de 21 a 242
Se demoró 0.84 segundos

En el caso 2 hay 800 datos y reduciremos la dimensión de los datos de 101 a 326
Se demoró 8.91 segundos

En el caso 3 hay 2000 datos y reduciremos la dimensión de los datos de 301 a 363
Se demoró 77.80 segundos

En el caso 4 hay 3000 datos y reduciremos la dimensión de los datos de 501 a 380
Se demoró más de 100 segundos

En el caso 5 hay 5000 datos y reduciremos la dimensión de los datos de 1001 a 401
Se demoró más de 100 segundos



In [359]:
for i in range(1, 6):
    X, Y = leer(i)
    k = floor(0.8*X.shape[1])
    A = np.random.randn(k, X.shape[1])
    newX = (1.0/sqrt(k))*np.dot(X, A.T)
    print(f"En el caso {i} hay {X.shape[0]} datos y reduciremos la dimensión de los datos de {X.shape[1]} a {k}")
    try:
        tstart = time.time()
        w = softSVMsolver(newX, Y)
        tfinish = time.time()
        print(f"Se demoró {(tfinish - tstart):.2f} segundos")
    except FunctionTimedOut:
        print(f"Se demoró más de {timelimit} segundos")
    print()

En el caso 1 hay 100 datos y reduciremos la dimensión de los datos de 21 a 16
Se demoró 0.58 segundos

En el caso 2 hay 800 datos y reduciremos la dimensión de los datos de 101 a 80
Se demoró 6.65 segundos

En el caso 3 hay 2000 datos y reduciremos la dimensión de los datos de 301 a 240
Se demoró 68.32 segundos

En el caso 4 hay 3000 datos y reduciremos la dimensión de los datos de 501 a 400
Se demoró más de 100 segundos

En el caso 5 hay 5000 datos y reduciremos la dimensión de los datos de 1001 a 800
Se demoró más de 100 segundos



Si hay una ganancia de velocidad, puesto que ahora el caso número 3 si es capaz de correr para la versión (Soft-SVM) y basta ver cada ejemplo para notar la mejora en la velocidad. Sin embargo, la mejora de la velocidad es de aproximadamente al doble y no es suficiente para compensar que hay que probar al menos unas 10 matrices diferentes para recién ser capaces de obtener una respuesta. En este sentido, la verdad es que no ganamos nada y considero que es incluso más lento que el anterior el algoritmo y que nos serviría para conjuntos de 10 veces menos tamaño en ambos lados que el anterior.

Un punto importante a considerar es que desconozco como fueron obtenidos los datos. Quizás en casos donde los datos tienen correlación esta idea de reducción de la dimensionalidad pueda tener más ventajas. Se me ocurre que por ejemplo en una base de datos de salud, puedes juntar ciertos datos en uno solo ya que son muy correlacionados.

Para el dual de (Hard-SVM) tenemos que
\begin{aligned}
\max_{\alpha \in \mathbb{R}^n} \quad & \sum_{i=1}^n \alpha_i - \frac{1}{2} \sum_{i=1}^n \sum_{k=1}^n \alpha_i \alpha_k y_i y_k \langle x_i, x_k \rangle \\
\text{s.t:} \quad & \alpha_i \geq 0 \quad \forall i\in[n]
\end{aligned}

Por lo que se tiene que $P = (Y_iY_k \langle X_i, X_k \rangle)_{i,k\in[n]}$, $q = \vec{1}_d$, $G = -I_n$, $h = \vec{0}_d$



In [363]:
@func_set_timeout(timelimit)
def dualhardSVMsolver(X, Y):
    n, d = X.shape
    P = (np.dot(X, X.T)*Y[:, np.newaxis])*Y[np.newaxis, :]
    q = -np.ones(n, dtype = float)
    G = -np.eye(n, dtype = float)
    h = np.zeros(n, dtype = float)
    alpha = solver(P, q, G, h)
    if alpha is None:
        return None
    w = np.sum(alpha[:, np.newaxis] * Y[:, np.newaxis] * X, axis = 0)
    return w

Ahora probemos nuestra implementación del dual de (Hard-SVM)

In [364]:
for i in range(1, 6):
    X, Y = leer(i)
    print(f"Estamos resolviendo el caso {i} que tiene {X.shape[0]} datos y {X.shape[1] - 1} características")
    tstart = time.time()
    try:
        w = dualhardSVMsolver(X, Y)
        tfinish = time.time()
        print(f"Se demoró {(tfinish - tstart):.2f} segundos y ", end = '')
        if w is None:
            print("no existe un separador afín")
        else:
            print(f"encontró un separador de margen {(1/np.dot(w, w)):.2f}")
    except FunctionTimedOut:
        print(f"Se está demorando más de {timelimit} segundos así que abortaremos el cálculo")
    print()

Estamos resolviendo el caso 1 que tiene 100 datos y 20 características
Se demoró 0.05 segundos y encontró un separador de margen 0.83

Estamos resolviendo el caso 2 que tiene 800 datos y 100 características
Se demoró 0.35 segundos y no existe un separador afín

Estamos resolviendo el caso 3 que tiene 2000 datos y 300 características
Se demoró 4.31 segundos y encontró un separador de margen 0.74

Estamos resolviendo el caso 4 que tiene 3000 datos y 500 características
Se demoró 16.82 segundos y no existe un separador afín

Estamos resolviendo el caso 5 que tiene 5000 datos y 1000 características
Se demoró 50.15 segundos y encontró un separador de margen 1.27



Como era de esperarse obtuvo los mismos resultados. Sin embargo, la implementación sobre el dual de (Hard-SVM) fue bastante más lenta que la del primal de (Hard-SVM)

Para el dual de (Soft-SVM) comencemos desde el problema primal
\begin{aligned}
&\min_{w \in \mathbb{R}^d} &\frac{\lambda}{2} \lVert w \rVert_2^2+ \sum_{i=1}^n \xi_i &\\
&\text{s.a. } &-Y_i\langle X_i, w \rangle \leq \xi_i - 1 & \ \ \forall i \in [n] \\
& & -\xi_i \leq 0 & \ \ \forall i \in [n]
\end{aligned}

Escribamos el langrangeano de esta función

$$\mathcal{L}(w, \xi, \alpha, \beta) = \frac{\lambda}{2} \lVert w \rVert_2^2+ \sum_{i=1}^n \xi_i + \sum_{i=1}^n \alpha_i(1 - Y_i\langle X_i, w\rangle - \xi_i) + \sum_{i=1}^n\beta_i(-\xi_i)$$

Simplificando un poco
$$\mathcal{L}(w, \xi, \alpha, \beta) = \frac{\lambda}{2} \lVert w \rVert_2^2 + \sum_{i=1}^n \alpha_i - \sum_{i=1}^n \alpha_i Y_i\langle X_i, w\rangle + \sum_{i=1}^n \xi_i(1 - \alpha_i - \beta_i)$$

Tenemos que $\frac{\partial \mathcal{L}}{\partial w} = \lambda w - \sum_{i=1}^n\alpha_iX_iY_i$. Al igualarlo a cero para encontrar el óptimo obtenemos la relación

$$w = \frac{1}{\lambda}\sum_{i=1}^n\alpha_iX_iY_i$$

Tenemos que $\frac{\partial \mathcal{L}}{\partial \xi} = \vec{1} - \alpha - \beta$. Al igualarlo a cero para encontrar el óptimo obtenemos la relación

$$\vec{1} - \alpha = \beta$$

Reemplazando estos valores en el langrangeano obtenemos que en el óptimo nos interesa estudiar la función
$$\mathcal{L} = \frac{\lambda}{2} \lVert \frac{1}{\lambda}\sum_{i=1}^n\alpha_iX_iY_i \rVert_2^2  + \sum_{i=1}^n \alpha_i - \sum_{i=1}^n \alpha_i Y_i\langle X_i, \frac{1}{\lambda}\sum_{i=1}^n\alpha_iX_iY_i\rangle + \sum_{i=1}^n \xi_i(1 - \alpha_i - (1 - \alpha_i))$$
$$\mathcal{L} = \sum_{i=1}^n \alpha_i + \frac{1}{2\lambda} \sum_{i=1}^n\sum_{k=1}^n \alpha_i \alpha_k Y_i Y_k \langle X_i, X_k \rangle - \frac{1}{\lambda} \sum_{i=1}^n\sum_{k=1}^n \alpha_i \alpha_k Y_i Y_k \langle X_i, X_k \rangle$$
$$\mathcal{L} = \sum_{i=1}^n \alpha_i - \frac{1}{2\lambda} \sum_{i=1}^n\sum_{k=1}^n \alpha_i \alpha_k Y_i Y_k \langle X_i, X_k \rangle$$

Pero recordemos que además impusimos la restricción
$$1 - \alpha_i = \beta_i$$
Pero como $\beta_i$ no aparece en ningún otro lugar, podemos hacer que tome cualquier valor mayor o igual a cero, de lo que se concluye que
$$1 - \alpha_i \geq 0 \rightarrow \alpha_i \leq 1$$

Por lo que el dual de (Soft-SVM) corresponde a
\begin{aligned}
\max_{\alpha \in \mathbb{R}^n} \quad & \sum_{i=1}^n \alpha_i - \frac{1}{2\lambda} \sum_{i=1}^n \sum_{j=1}^n \alpha_i \alpha_j y_i y_j \langle x_i, x_j \rangle \\
\text{s.t.} \quad & 0 \leq \alpha_i \leq 1, \quad \forall i \in [n], \\
\end{aligned}

¡Implementémoslo!

In [365]:
def dualsoftSVM(X, Y, Lambda):
    n, d = X.shape
    P = (1/Lambda)*(np.dot(X, X.T)*Y[:, np.newaxis])*Y[np.newaxis, :]
    q = -np.ones(n, dtype = float)
    G = np.zeros((2*n, n), dtype = float)
    G[:n, :n] = -np.eye(n, dtype = float)
    G[n:2*n, :n] = np.eye(n, dtype = float)
    h = np.zeros(2*n, dtype = float)
    h[n:2*n] = np.ones(n, dtype = float)
    alpha = solver(P, q, G, h)
    w = (1/Lambda)*np.sum(alpha[:, np.newaxis] * Y[:, np.newaxis] * X, axis = 0)
    return w

@func_set_timeout(timelimit)
def dualsoftSVMsolver(X, Y):
    n, d = X.shape
    X1, X2, X3, Y1, Y2, Y3 = splitter(X, Y)
    best = n+1
    wbest = np.zeros(d)
    for exp in range(0, 11):
        Lambda = 2**(-exp)
        w = dualsoftSVM(X1, Y1, Lambda)
        v = (Lambda/2.0)*np.dot(w, w) + sum(max(0.0, 1.0 - np.dot(w, X2[i])*Y2[i]) for i in range(len(X2)))
        if v < best:
            best = v
            wbest = w
    risk = 100*(sum(1 for i in range(len(X3)) if Y3[i] *np.dot(wbest, X3[i]) < 1))/len(X3)
    return wbest, risk

Y finalmente probemos nuestra implementación del dual de (Soft-SVM)

In [366]:
for i in range(1, 6):
    X, Y = leer(i)
    print(f"Estamos resolviendo el caso {i} que tiene {X.shape[0]} datos y {X.shape[1] - 1} características")
    tstart = time.time()
    try:
        w, risk = dualsoftSVMsolver(X, Y)
        tfinish = time.time()
        print(f"Se demoró {(tfinish - tstart):.2f} segundos y ", end = '')
        print(f"encontró un separador de margen {(1/np.dot(w, w)):.2f} y en los datos de testing acertó un {risk}%")
    except FunctionTimedOut:
        print(f"Se está demorando más de {timelimit} segundos así que abortaremos el cálculo")
    print()

Estamos resolviendo el caso 1 que tiene 100 datos y 20 características
Se demoró 0.49 segundos y encontró un separador de margen 1.75 y en los datos de testing acertó un 50.0%

Estamos resolviendo el caso 2 que tiene 800 datos y 100 características
Se demoró 5.60 segundos y encontró un separador de margen 4.54 y en los datos de testing acertó un 41.25%

Estamos resolviendo el caso 3 que tiene 2000 datos y 300 características
Se demoró 55.01 segundos y encontró un separador de margen 1.14 y en los datos de testing acertó un 18.5%

Estamos resolviendo el caso 4 que tiene 3000 datos y 500 características
Se está demorando más de 100 segundos así que abortaremos el cálculo

Estamos resolviendo el caso 5 que tiene 5000 datos y 1000 características
Se está demorando más de 100 segundos así que abortaremos el cálculo



Para el (Soft-SVM) podemos observar que la implementación del dual fue bastante más rápido, siendo cercano al doble más rápido que la implementación del algoritmo del primal